In [1]:
import pandas as pd
import numpy as np 
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier

Importing the training dataset

In [2]:
train = pd.read_csv('E:/Arvind/Kaggle/Titanic Machine Learning from Disaster/Input/train.csv')

In [3]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [8]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


EDA on each column individually and then vs the dependent

1) Pclass

In [11]:
train['Pclass'].value_counts(normalize = True).sort_index()

1    0.242424
2    0.206510
3    0.551066
Name: Pclass, dtype: float64

In [4]:
train.groupby(['Pclass'])['Survived'].mean()

Pclass
1    0.629630
2    0.472826
3    0.242363
Name: Survived, dtype: float64

2) Sex          

In [13]:
train['Sex'].value_counts(normalize = True).sort_index()

female    0.352413
male      0.647587
Name: Sex, dtype: float64

In [14]:
train.groupby('Sex')['Survived'].mean()

Sex
female    0.742038
male      0.188908
Name: Survived, dtype: float64

3) Age
a) imputing missing values with median
b) Age - by bucketing 
c) Age - checking correlation with the dependent

In [4]:
train['Age_FillNA'] = train['Age'].fillna(train['Age'].median())

In [5]:
pd.qcut(train['Age_FillNA'],4).value_counts(normalize = True).sort_index()

(0.419, 22.0]    0.259259
(22.0, 28.0]     0.345679
(28.0, 35.0]     0.151515
(35.0, 80.0]     0.243547
Name: Age_FillNA, dtype: float64

In [22]:
train.groupby(pd.qcut(train['Age_FillNA'],4))['Survived'].mean()

Age_FillNA
(0.419, 22.0]    0.424242
(22.0, 28.0]     0.331169
(28.0, 35.0]     0.437037
(35.0, 80.0]     0.382488
Name: Survived, dtype: float64

In [24]:
train[['Age_FillNA','Survived']].corr()

,Age_FillNA,Survived
Age_FillNA,1.00000,-0.06491
Survived,-0.06491,1.00000


4) SibSp        

In [26]:
train['SibSp'].value_counts(normalize = True).sort_index()

0    0.682379
1    0.234568
2    0.031425
3    0.017957
4    0.020202
5    0.005612
8    0.007856
Name: SibSp, dtype: float64

In [27]:
train.groupby('SibSp')['Survived'].mean()

SibSp
0    0.345395
1    0.535885
2    0.464286
3    0.250000
4    0.166667
5    0.000000
8    0.000000
Name: Survived, dtype: float64

5) We will combine SibSp and parch as a feature

In [29]:
train['RelationsOnboard'] = train['SibSp'] + train['Parch']

In [30]:
train['RelationsOnboard'].value_counts(normalize = True).sort_index()

0     0.602694
1     0.180696
2     0.114478
3     0.032548
4     0.016835
5     0.024691
6     0.013468
7     0.006734
10    0.007856
Name: RelationsOnboard, dtype: float64

In [31]:
train.groupby('RelationsOnboard')['Survived'].mean()

RelationsOnboard
0     0.303538
1     0.552795
2     0.578431
3     0.724138
4     0.200000
5     0.136364
6     0.333333
7     0.000000
10    0.000000
Name: Survived, dtype: float64

6) Fare
a) check correlation with class and survived
b) EDA by bucketing

In [34]:
train[['Fare','Pclass','Survived']].corr()

,Fare,Pclass,Survived
Fare,1.000000,-0.549500,0.257307
Pclass,-0.549500,1.000000,-0.338481
Survived,0.257307,-0.338481,1.000000


In [36]:
pd.qcut(train['Fare'],4).value_counts(normalize = True).sort_index()

(-0.001, 7.91]     0.250281
(7.91, 14.454]     0.251403
(14.454, 31.0]     0.249158
(31.0, 512.329]    0.249158
Name: Fare, dtype: float64

In [37]:
train.groupby(pd.qcut(train['Fare'],4))['Survived'].mean()

Fare
(-0.001, 7.91]     0.197309
(7.91, 14.454]     0.303571
(14.454, 31.0]     0.454955
(31.0, 512.329]    0.581081
Name: Survived, dtype: float64

7) Embarked
a) Impute missing values
b) EDA

In [41]:
train['Embarked_FillNA'] = train['Embarked'].fillna(train['Embarked'].mode())

In [44]:
train['Embarked_FillNA'].value_counts(normalize = True).sort_index()

C    0.188976
Q    0.086614
S    0.724409
Name: Embarked_FillNA, dtype: float64

In [45]:
train.groupby('Embarked_FillNA')['Survived'].mean()

Embarked_FillNA
C    0.553571
Q    0.389610
S    0.336957
Name: Survived, dtype: float64

Our final features are:
1) Pclass
2) Sex
3) Age
4) Relations Onboard
5) Embarked

1, 2 and 5 need to be encoded

In [53]:
train = pd.concat((train,pd.get_dummies(train['Pclass'],prefix = 'Pclass' )),axis=1)

In [58]:
train = pd.concat((train,pd.get_dummies(train['Sex'],prefix = 'Sex' )),axis=1)

In [59]:
train = pd.concat((train,pd.get_dummies(train['Embarked_FillNA'],prefix = 'Embarked' )),axis=1)

In [60]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,...,RelationsOnboard,Embarked_FillNA,Pclass_1,Pclass_2,Pclass_3,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,...,1,S,0,0,1,0,1,0,0,1
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,...,1,C,1,0,0,1,0,1,0,0
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,...,0,S,0,0,1,1,0,0,0,1
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,...,1,S,1,0,0,1,0,0,0,1
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,...,0,S,0,0,1,0,1,0,0,1


In [61]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 23 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   PassengerId       891 non-null    int64  
 1   Survived          891 non-null    int64  
 2   Pclass            891 non-null    int64  
 3   Name              891 non-null    object 
 4   Sex               891 non-null    object 
 5   Age               714 non-null    float64
 6   SibSp             891 non-null    int64  
 7   Parch             891 non-null    int64  
 8   Ticket            891 non-null    object 
 9   Fare              891 non-null    float64
 10  Cabin             204 non-null    object 
 11  Embarked          889 non-null    object 
 12  Age_FillNA        891 non-null    float64
 13  RelationsOnboard  891 non-null    int64  
 14  Embarked_FillNA   889 non-null    object 
 15  Pclass_1          891 non-null    uint8  
 16  Pclass_2          891 non-null    uint8  
 1

Test data import

In [62]:
test = pd.read_csv('E:/Arvind/Kaggle/Titanic Machine Learning from Disaster/Input/test.csv')

In [63]:
test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [64]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Pclass       418 non-null    int64  
 2   Name         418 non-null    object 
 3   Sex          418 non-null    object 
 4   Age          332 non-null    float64
 5   SibSp        418 non-null    int64  
 6   Parch        418 non-null    int64  
 7   Ticket       418 non-null    object 
 8   Fare         417 non-null    float64
 9   Cabin        91 non-null     object 
 10  Embarked     418 non-null    object 
dtypes: float64(2), int64(4), object(5)
memory usage: 36.0+ KB


Impute missing values in age

In [65]:
test['Age_FillNA'] = test['Age'].fillna(test['Age'].median())

Creating Relations Onboard column

In [66]:
test['RelationsOnboard'] = test['SibSp'] + test['Parch']

Our final features are:
1) Pclass
2) Sex
3) Age
4) Relations Onboard
5) Embarked

1, 2 and 5 need to be encoded

In [68]:
test = pd.concat((test,pd.get_dummies(test['Pclass'],prefix = 'Pclass' )),axis=1)

In [69]:
test = pd.concat((test,pd.get_dummies(test['Sex'],prefix = 'Sex' )),axis=1)

In [70]:
test = pd.concat((test,pd.get_dummies(test['Embarked'],prefix = 'Embarked' )),axis=1)

In [71]:
test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,...,Age_FillNA,RelationsOnboard,Pclass_1,Pclass_2,Pclass_3,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,...,34.5,0,0,0,1,0,1,0,1,0
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,...,47.0,1,0,0,1,1,0,0,0,1
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,...,62.0,0,0,1,0,0,1,0,1,0
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,...,27.0,0,0,0,1,0,1,0,0,1
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,...,22.0,2,0,0,1,1,0,0,0,1


In [72]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   PassengerId       418 non-null    int64  
 1   Pclass            418 non-null    int64  
 2   Name              418 non-null    object 
 3   Sex               418 non-null    object 
 4   Age               332 non-null    float64
 5   SibSp             418 non-null    int64  
 6   Parch             418 non-null    int64  
 7   Ticket            418 non-null    object 
 8   Fare              417 non-null    float64
 9   Cabin             91 non-null     object 
 10  Embarked          418 non-null    object 
 11  Age_FillNA        418 non-null    float64
 12  RelationsOnboard  418 non-null    int64  
 13  Pclass_1          418 non-null    uint8  
 14  Pclass_2          418 non-null    uint8  
 15  Pclass_3          418 non-null    uint8  
 16  Sex_female        418 non-null    uint8  
 1

Fit RF Classifier with Grid Search

In [2]:
RF = RandomForestClassifier(n_estimators=300)

NameError: name 'RandomForestClassifier' is not defined

In [91]:
param_grid = { "criterion" : ["gini", "entropy"], "min_samples_leaf" : [1, 5, 10], "min_samples_split" : [1,5,10]}

In [92]:
gs = GridSearchCV(estimator=RF, param_grid=param_grid, cv=3)

In [93]:
gs.fit(train[['Age_FillNA', 'RelationsOnboard','Pclass_1','Pclass_2','Pclass_3','Sex_female','Sex_male','Embarked_C','Embarked_Q','Embarked_S']],train['Survived'])

C:\Users\admin\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:548: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\admin\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 531, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\admin\anaconda3\lib\site-packages\sklearn\ensemble\_forest.py", line 386, in fit
    trees = Parallel(n_jobs=self.n_jobs, verbose=self.verbose,
  File "C:\Users\admin\anaconda3\lib\site-packages\joblib\parallel.py", line 1048, in __call__
    if self.dispatch_one_batch(iterator):
  File "C:\Users\admin\anaconda3\lib\site-packages\joblib\parallel.py", line 866, in dispatch_one_batch
    self._dispatch(tasks)
  File "C:\Users\admin\anaconda3\lib\site-packages\joblib\parallel.py", line 784, in _dispatch
    job = self._backend.apply_async(batch, callback=cb)

C:\Users\admin\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:548: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\admin\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 531, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\admin\anaconda3\lib\site-packages\sklearn\ensemble\_forest.py", line 386, in fit
    trees = Parallel(n_jobs=self.n_jobs, verbose=self.verbose,
  File "C:\Users\admin\anaconda3\lib\site-packages\joblib\parallel.py", line 1048, in __call__
    if self.dispatch_one_batch(iterator):
  File "C:\Users\admin\anaconda3\lib\site-packages\joblib\parallel.py", line 866, in dispatch_one_batch
    self._dispatch(tasks)
  File "C:\Users\admin\anaconda3\lib\site-packages\joblib\parallel.py", line 784, in _dispatch
    job = self._backend.apply_async(batch, callback=cb)

C:\Users\admin\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:548: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\admin\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 531, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\admin\anaconda3\lib\site-packages\sklearn\ensemble\_forest.py", line 386, in fit
    trees = Parallel(n_jobs=self.n_jobs, verbose=self.verbose,
  File "C:\Users\admin\anaconda3\lib\site-packages\joblib\parallel.py", line 1048, in __call__
    if self.dispatch_one_batch(iterator):
  File "C:\Users\admin\anaconda3\lib\site-packages\joblib\parallel.py", line 866, in dispatch_one_batch
    self._dispatch(tasks)
  File "C:\Users\admin\anaconda3\lib\site-packages\joblib\parallel.py", line 784, in _dispatch
    job = self._backend.apply_async(batch, callback=cb)

GridSearchCV(cv=3, estimator=RandomForestClassifier(),
             param_grid={'criterion': ['gini', 'entropy'],
                         'min_samples_leaf': [1, 5, 10],
                         'min_samples_split': [1, 5, 10]})

In [97]:
print(gs.best_score_)
print(gs.best_params_)
# print(gs.cvresults)

0.8215488215488215
{'criterion': 'entropy', 'min_samples_leaf': 5, 'min_samples_split': 5}


In [98]:
submission = gs.predict(test[['Age_FillNA', 'RelationsOnboard','Pclass_1','Pclass_2','Pclass_3','Sex_female','Sex_male','Embarked_C','Embarked_Q','Embarked_S']])

In [99]:
pd.DataFrame(submission).head()

,0
0,0
1,0
2,0
3,0
4,1


In [100]:
submission = pd.concat((test['PassengerId'],pd.DataFrame(submission,columns = ['Survived'])),axis =1)

In [101]:
submission.head()

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,1


In [102]:
submission.to_csv('E:/Arvind/Kaggle/Titanic Machine Learning from Disaster/Output/submission_V2.csv')